<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/4-Evaluating%20LLMs/4_3_evaluating_rag_giskard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.3-Evaluating LLMs with other LLMs.
  </h2>
    <h3>Evaluating a RAG solution with Giskard</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra)

____________
Models: OpenAI.

Colab Environment: CPU-High RAM

Keys:
* LangChain.
* RAG.
* Giskard.
___________

This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (<a href="https://link.springer.com/book/10.1007/979-8-8688-0515-8">Apress</a>).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

In the final part of this notebook, you will find the necessary code to evaluate the suitability of the responses provided by the Agent using Giskard.

#Installing libraries & Loading Dataset

In [1]:
!pip install -q langchain==0.1.14
!pip install -q langchain-openai==0.1.1
!pip install langchainhub==0.1.15
#!pip install --upgrade -q tiktoken==0.5.2
!pip install -q datasets==2.16.1
!pip install -q chromadb==0.4.22
!pip install -q openai==1.30.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/38

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [2]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [3]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [4]:
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [5]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [6]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [7]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [10]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. 
                
Following a few days of recovery, a second phase of illness may occur. Symptoms may consist of meningitis (fever, headache, stiff neck, etc.), encephalitis (drowsiness, confusion, sensory disturbances, and/or motor abnormalities, such as paralysis), or meningoencephalitis (inflammation 

### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [11]:
from getpass import getpass
import os
if not 'OPENAI_API_KEY' in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [12]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    #openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [14]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(#openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [17]:
qa.invoke("What is the main symptom of LCM?")

{'query': 'What is the main symptom of LCM?',
 'result': ' The main symptom of LCM is a biphasic febrile illness, which includes symptoms such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting.'}

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [18]:
from langchain.agents import Tool, AgentExecutor
DESCRIPTION = 'use this tool when answering medical knowledge queries to get more information about the topic'
#DESCRIPTION = 'Use this tool if any part of the QUERY or CHAT HISTORY history discusses concepts related to medicine or diseases to get more information about the topic'

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            DESCRIPTION
        )
    )
]

In [19]:
from langchain.agents import initialize_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    #agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    prompt=prompt,
    #verbose=True,
    #max_iterations=3,
    #early_stopping_method='generate'
)

In [20]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               #early_stopping_method='generate',
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [21]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Area of square I don't know.Do I need to use a tool? No
Final Answer: The area of a square with sides of length 2 is 4.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with sides of length 2 is 4.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [22]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Clark Kent is the secret identity of the superhero Superman.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2'),
  AIMessage(content='The area of a square with sides of length 2 is 4.')],
 'output': 'Clark Kent is the secret identity of the superhero Superman.'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [23]:
 agent_executor.memory.clear()

In [24]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of contaminated food or wound.Do I need to use a tool? No
Final Answer: To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use electromyography (EMG) to test muscle function and nerve conduction. It is important to seek medical attention immediately if botulism is suspected.

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnose?',
 'chat_history': [],
 'output': 'To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use electromyography (EMG) to test muscle function and nerve conduction. It is important to seek medical attention immediately if botulism is suspected.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [25]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Yes, botulism is a serious illness that can lead to paralysis and even death if left untreated. It is important to seek medical attention immediately if it is suspected.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?'),
  AIMessage(content='To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use electromyography (EMG) to test muscle function and nerve conduction. It is important to seek medical attention immediately if botulism is suspected.')],
 'output': 'Yes, botulism is a serious illness that can lead to paralysis and even death if left untreated. It is important to seek medical attention immediately if it is suspected.'}

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

## Evaluating the solution with Giskard

Install and Load the libraries.

In [35]:
!pip install -q giskard[llm]==2.15.1
from giskard.rag import KnowledgeBase, generate_testset, evaluate

Is necesary to create a Dataframe with just the column containing the information used to create the RAG system.

In [36]:
import pandas as pd
df_giskard = pd.DataFrame([d.page_content for d in df_document], columns=["text"])
df_giskard.head()

,text
0,LCMV infections can occur after exposure to fr...
1,LCMV is most commonly recognized as causing ne...
2,Individuals of all ages who come into contact ...
3,"During the first phase of the disease, the mos..."
4,"Aseptic meningitis, encephalitis, or meningoen..."


Using the information from the dataset, we ask Giskard to create a Knowledge Base, which is nothing more than a set of questions along with their respective answers. Both the questions and answers are generated by OpenAI's most advanced model, which is why it requires our OpenAI key to be provided.

In [37]:
kb_giskard = KnowledgeBase(df_giskard)

In [38]:
kb_giskard

In [39]:
# The more questions you generate, the more you will be charged.
test_questions = generate_testset(
    kb_giskard,
    num_questions=10,
    agent_description="Medical assistant for diagnosis and treatment support.",
)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Found 7 topics in the knowledge base.


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

In [40]:
df_test_questions = test_questions.to_pandas()

In [41]:
df_test_questions.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
38c7c5f3-4330-4f13-a29a-c20afb0ca114,What are the characteristics of seafood contam...,The seafood contaminated with marine toxins fr...,Document 16: Marine toxins are naturally occur...,[],"{'question_type': 'simple', 'seed_document_id'..."
f0e2a8be-bb6c-4156-8a4c-69c178824e60,What are the laboratory methods used to diagno...,Antigen-capture enzyme-linked immunosorbent as...,Document 58: Clinical diagnosis could be diffi...,[],"{'question_type': 'simple', 'seed_document_id'..."
756adb66-4937-4a34-95f6-f2c7efc66378,Could you enumerate the typical manifestations...,The classic symptoms of botulism include doubl...,Document 84: Botulism can result in death due ...,[],"{'question_type': 'complex', 'seed_document_id..."
a22360a6-fb60-49bb-8fc3-fff4154e9039,Could you elucidate on the origin of the Trich...,"Trichuriasis infection is caused by Whipworm, ...",Document 11: Whipworm (Trichuris trichiura) is...,[],"{'question_type': 'complex', 'seed_document_id..."
ca45a942-afa3-485e-81f0-1f70f9db3d18,What are the initial symptoms that may appear ...,The disease presents initially with non-specif...,"Document 56: Based on limited information, aft...",[],"{'question_type': 'distracting element', 'seed..."


In [42]:
df_test_questions['question'].iloc[1]


'What are the laboratory methods used to diagnose Marburg Hemorrhagic Fever?'

In [43]:
df_test_questions['reference_answer'].iloc[1]


'Antigen-capture enzyme-linked immunosorbent assay (ELISA) testing, polymerase chain reaction (PCR), and IgM-capture ELISA can be used to confirm a case of Marburg HF within a few days of symptom onset. Virus isolation may also be performed but should only be done in a high containment laboratory with good laboratory practices. The IgG-capture ELISA is appropriate for testing persons later in the course of disease or after recovery. In deceased patients, immunohistochemistry, virus isolation, or PCR of blood or tissue specimens may be used to diagnose Marburg HF retrospectively.'

In [44]:
df_test_questions['reference_context'].iloc[1]

'Document 58: Clinical diagnosis could be difficult due to similarities between AVHF, Crimean-Congo Hemorrhagic fever (CCHF), and Rift Valley fever (RVF), which occur in similar geographic areas. Laboratory diagnosis of AHF can be made in the early stage of the illness by molecular detection by PCR or virus isolation from blood. Later, serologic testing using enzyme-linked immunosorbent serologic assay (ELISA) can be performed.\n\nDocument 90: Many of the signs and symptoms of Marburg hemorrhagic fever are similar to those of other more frequent infectious diseases, such as malaria or typhoid fever, making diagnosis of the disease difficult. This is especially true if only a single case is involved. \n \nHowever, if a person has the early symptoms of Marburg HF and there is reason to believe that Marburg HF should be considered, the patient should be isolated and public health professionals notified. Samples from the patient can then be collected and tested to confirm infection. \n \nA

A function is created that will be called from Giskard's evaluate function. This function receives the question and returns the agent's response.

In [45]:
def use_agent(question, history=None):
  answer=agent_executor.invoke({"input": question})
  return answer['output']

In [46]:
agent_executor.memory.clear()

In [47]:
report = evaluate(use_agent, testset=test_questions, knowledge_base=kb_giskard)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Seafood contamination Seafood contamination can occur due to the presence of marine toxins, which are naturally occurring chemicals that can contaminate certain seafood. These toxins can cause illness in humans when consumed, even if the seafood looks, smells, and tastes normal. Health departments monitor and test shellfish to assess the risk of contamination and may prohibit harvesting during periods of risk. It is important to notify public health departments of any cases of marine toxin poisoning so they can investigate and prevent further illnesses. Consumers should also freeze any uneaten portions of seafood in case they need to be tested. Do I need to use a tool? No
Final Answer: Some characteristics of seafood contaminated with marine toxins include the presence of naturally occurring chemicals, potential illness when consumed, and the need for monitoring and testing b

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [48]:
# Summary with the results.
report.correctness_by_question_type()

,correctness
question_type,
complex,0.5
conversational,0.0
distracting element,1.0
double,0.0
simple,0.5
situational,0.5


In [49]:
report.correctness_by_topic()

,correctness
topic,
Disease Diagnosis and Treatment,0.00
Foodborne Illnesses and Poisoning,0.25
Infectious Diseases and Prevention,0.00
Others,1.00
Parasitic Infections and Treatments,1.00


In [50]:
report.component_scores()

,score
RAG Components,
GENERATOR,0.500000
RETRIEVER,0.750000
REWRITER,0.333333
ROUTING,1.000000
KNOWLEDGE_BASE,0.000000


In [51]:
# Obtaining the incorrect answers
failures = report.get_failures()[:2]
failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
38c7c5f3-4330-4f13-a29a-c20afb0ca114,What are the characteristics of seafood contam...,The seafood contaminated with marine toxins fr...,Document 16: Marine toxins are naturally occur...,[],"{'question_type': 'simple', 'seed_document_id'...",Some characteristics of seafood contaminated w...,False,The agent's answer does not mention that seafo...
756adb66-4937-4a34-95f6-f2c7efc66378,Could you enumerate the typical manifestations...,The classic symptoms of botulism include doubl...,Document 84: Botulism can result in death due ...,[],"{'question_type': 'complex', 'seed_document_id...",Some typical manifestations of botulism includ...,False,The agent's answer does not fully match the gr...


In [52]:
# Giskard explains the reasons why it considers the answers to be incorrect.
failures['correctness_reason'].iloc[1]

"The agent's answer does not fully match the ground truth. The agent did not mention all the symptoms of botulism such as double vision, blurred vision, drooping eyelids, slurred speech, dry mouth, and specific symptoms in infants. The timeline provided by the agent is correct, but the answer is incomplete."


# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---